In [1]:
import sys
sys.path.append('..')

In [2]:
import sqlite3
import pandas as pd
from copy import deepcopy
from os.path import join
import gc

In [3]:
from os.path import join
from aequilibrae import Project
from aequilibrae.paths import TrafficAssignment, TrafficClass
from aequilibrae.paths.select_link import SelectLink
import logging
import sys

import numpy as np

In [4]:
pth = 'D:/GDrive/DATA/Pedro/Professional/Consulting/2021/01-01 - Network criticality analysis (UArk)/DATA/simple_model'

proj = Project()
proj.open(pth)
net = proj.network

In [ ]:
%%time
mat_list = proj_matrices.list()

modes = net.modes
modes.all_modes()

nodes = net.nodes

for n in [4701, 4702, 4703]:
    nd = nodes.get(n)
    nd.is_centroid = 0
    nd.save()

net.build_graphs(modes=['T'])
truckGraph = net.graphs['T']
net.build_graphs(modes=['c'])
carGraph = net.graphs['c']

In [ ]:
%%time
# Link exclusions

exclude_from_passenger = [x[0] for x in proj.conn.execute("select link_id from links where exclusionset IN ('TruckOnly', 'HOV2', 'HOV3')")]

exclude_from_truck = [x[0] for x in proj.conn.execute("select link_id from links where exclusionset IN ('PassengerOnly', 'HOV2', 'HOV3')")]


graph = truckGraph
set1 = graph.network[(graph.network.builtyear > 2010) | (graph.network.removedyear < 2010)].link_id.to_list()
set2 = graph.network[(graph.network.mode_code <10) | (graph.network.mode_code >11)].link_id.to_list()
exclude_from_passenger.extend(set1 + set2)
exclude_from_truck.extend(set1 + set2)

truckGraph.exclude_links(exclude_from_truck)
carGraph.exclude_links(exclude_from_passenger)

nodes = net.nodes
for n in [4701, 4702, 4703]:
    nd = nodes.get(n)
    nd.is_centroid = 1
    nd.save()

for graph in [carGraph, truckGraph]:
    graph.graph.alpha.fillna(0.15, inplace=True)
    graph.graph.beta.fillna(4.0, inplace=True)
    graph.graph.hov1tollcost.fillna(0, inplace=True)
    graph.graph.mttollcost.fillna(0, inplace=True)
    graph.graph.httollcost.fillna(0, inplace=True)
    for period in ['am', 'pm', 'md', 'nt']:
        graph.graph.loc[graph.graph.a_node == graph.graph.b_node, f"{period}_assncap_10"] = 1.0
        graph.graph.loc[graph.graph.a_node == graph.graph.b_node, f"tt_{period}_10"] = 0.001
ref_carGraph = deepcopy(carGraph)
ref_truckGraph = deepcopy(truckGraph)

## Baseline assignment

In [5]:
period= 'am'
proj_matrices = proj.matrices
proj_matrices = proj.matrices
carDemand = proj_matrices.get_matrix(f'{period.upper()}_omx')
carDemand.computational_view('AUTO')

lightTruckDemand = proj_matrices.get_matrix(f'{period.upper()}_omx')
lightTruckDemand.computational_view('COMMTRK')
lightTruckDemand.matrix_view = np.array(lightTruckDemand.matrix_view, np.float64)

heavyTruckDemand = proj_matrices.get_matrix(f'{period.upper()}_omx')
heavyTruckDemand.computational_view('HTRK')
heavyTruckDemand.matrix_view = np.array(heavyTruckDemand.matrix_view, np.float64)

In [ ]:

assig = TrafficAssignment()
# assig.procedure_id = f'{period}_baseline'
carClass = TrafficClass('car', carGraph, carDemand)
carClass.set_pce(1)
carClass.set_vot(0.2)
carClass.set_fixed_cost('hov1tollcost')

# The link exclusions for commercial trucks are actually the same as the ones for passenger cars, and not heavy trucks
lightTruckClass = TrafficClass('light_truck', carGraph, lightTruckDemand)
lightTruckClass.set_pce(1.5)
lightTruckClass.set_vot(0.5)
lightTruckClass.set_fixed_cost("mttollcost")

heavyTruckClass = TrafficClass('heavy_truck', truckGraph, heavyTruckDemand)
heavyTruckClass.set_pce(2.5)
heavyTruckClass.set_vot(1.0)
heavyTruckClass.set_fixed_cost("httollcost")

# The first thing to do is to add at list of traffic classes to be assigned
assig.set_classes([heavyTruckClass, carClass, lightTruckClass])
assig.set_vdf("BPR")  # This is not case-sensitive # Then we set the volume delay function

assig.set_vdf_parameters({"alpha": "alpha", "beta": "beta"})  # And its parameters

assig.set_time_field(f"tt_{period}_10")
assig.set_capacity_field(f"{period}_assncap_10")  # The capacity and free flow travel times as they exist in the graph

# And the algorithm we want to use to assign
assig.set_save_path_files(True)
assig.set_algorithm('msa')
assig.max_iter = 10
assig.rgap_target = 0.001

assig.execute()  # we then execute the assignment
assig.save_results(f'{period}__path_files')
baseline = assig.results()

In [6]:
carDemand.matrix_view = carDemand.matrix_view.reshape(6445, 6445, 1)
lightTruckDemand.matrix_view = lightTruckDemand.matrix_view.reshape(6445, 6445, 1)
heavyTruckDemand.matrix_view = heavyTruckDemand.matrix_view.reshape(6445, 6445, 1)


In [17]:
del sl
gc.collect()

568

In [7]:
sl_ids = [x for x in range(100)]
sl_ids = [0]

In [8]:
demand_mat = {"car": carDemand,"light_truck": lightTruckDemand, "heavy_truck": heavyTruckDemand}
res_name = f'{period}__path_files'
sl = SelectLink(res_name, demand_mat)

In [9]:
sl_result = sl.run_select_link_analysis(sl_ids)

FileNotFoundError: [Errno 2] No such file or directory: 'D:/GDrive/DATA/Pedro/Professional/Consulting/2021/01-01 - Network criticality analysis (UArk)/DATA/simple_model\\path_files\\2df734b2206746389f093145d8b88633\\iter3\\path_cc_car\\o0.feather'